In [1]:
%run "../LOS.ipynb"

In [28]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
from datetime import datetime

In [3]:
from re import sub
def name_handler(s):
    s = sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespace with a single dash
    s = sub(r"\s+", '_', s)
    return s



In [4]:
tested_hts = dreams_mastersheet[(dreams_mastersheet.timeOn_system == "required_Time_on")&(dreams_mastersheet.hts == "tested_on_given_date")&(dreams_mastersheet.newage_range != "10-17")&(dreams_mastersheet.newage_range != "25-29")&(dreams_mastersheet.newage_range != "not_valid_age")]

In [5]:
tested_hts_21 = tested_hts[tested_hts.fiscal_year != "Q3fy20-Q4fy20"]

## Nouvelles Filles HTS

In [6]:
hts_new = dreams_mastersheet[(dreams_mastersheet.timeOn_system == "required_Time_on")&(dreams_mastersheet.hts == "errata")&(dreams_mastersheet.newage_range != "10-17")&(dreams_mastersheet.newage_range != "25-29")&(dreams_mastersheet.newage_range != "not_valid_age")]

In [7]:
date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [8]:
hts_new.to_excel(f"verification_{date_du_jour}.xlsx",index=False,na_rep="NULL")

## Oldest

> verification

Pick the verification as the starting point

In [9]:
old_hts = pd.read_excel("verification.xlsx")

In [10]:
old_hts.code.count()
hts_new.code.count()
tested_hts.code.count()
tested_hts_21.code.count()

4799

4988

594

47

In [11]:
hts_new.date_entevyou.min().strftime("%Y-%m-%d")
hts_new.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-07'

'2021-07-03'

In [12]:
old_hts.date_entevyou.min().strftime("%Y-%m-%d")
old_hts.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-07'

'2021-06-15'

In [13]:
tested_hts.date_entevyou.min().strftime("%Y-%m-%d")
tested_hts.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-18'

'2021-06-05'

In [14]:
tested_hts_21.date_entevyou.min().strftime("%Y-%m-%d")
tested_hts_21.date_entevyou.max().strftime("%Y-%m-%d")

'2020-10-05'

'2021-06-05'

In [15]:
howmuch_htsNew_isin_htsold = hts_new[hts_new.code.isin(old_hts.code)]

In [16]:
howmuch_htsNew_isnot_htsold = hts_new[~hts_new.code.isin(old_hts.code)]

In [17]:
howmuch_tested_isin_htsold = tested_hts[tested_hts.code.isin(old_hts.code)]

In [18]:
howmuch_tested21_isin_htsold =  tested_hts_21[tested_hts_21.code.isin(old_hts.code)]

In [19]:
howmuch_htsNew_isin_htsold.code.count()
howmuch_htsNew_isnot_htsold.code.count()
howmuch_tested_isin_htsold.code.count()
howmuch_tested21_isin_htsold.code.count()

4750

238

16

13

In [20]:
howmuch_htsNew_isin_htsold.date_entevyou.min().strftime("%Y-%m-%d")
howmuch_htsNew_isin_htsold.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-07'

'2021-06-15'

In [21]:
howmuch_htsNew_isnot_htsold.date_entevyou.min().strftime("%Y-%m-%d")
howmuch_htsNew_isnot_htsold.date_entevyou.max().strftime("%Y-%m-%d")

'2020-05-29'

'2021-07-03'

In [22]:
howmuch_tested_isin_htsold.date_entevyou.min().strftime("%Y-%m-%d")
howmuch_tested_isin_htsold.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-18'

'2021-04-18'

In [23]:
howmuch_tested21_isin_htsold.date_entevyou.min().strftime("%Y-%m-%d")
howmuch_tested21_isin_htsold.date_entevyou.max().strftime("%Y-%m-%d")

'2020-10-06'

'2021-04-18'

## Report Component

In [24]:
hts_new_communes = []
for columns in hts_new.actual_commune.unique():
    globals()[f"hts_new_{name_handler(columns)}"] = hts_new[hts_new.actual_commune == columns]
    hts_new_communes.append(globals().get(f"hts_new_{name_handler(columns)}"))

In [25]:
new_communes_general = hts_new.actual_commune.unique().tolist()
hts_new_dict = {}
for columns, communes in zip(hts_new.actual_commune.unique(), new_communes_general):
    hts_new_dict[f"{name_handler(communes)}"] =  globals().get(f"hts_new_{name_handler(columns)}").code.count()
    
    
hts_new_df = pd.DataFrame.from_dict(hts_new_dict, orient="index", columns=None)
hts_new_df.columns = ["nombre_filles_pour_depistage_hts"]
hts_new_df

,nombre_filles_pour_depistage_hts
Dessalines,177
Kenscoff,787
Gressier,270
CapHaïtien,242
Milot,78
PlaineduNord,89
QuartierMorin,37
Limonade,119
Verrettes,553
Liancourt,40


In [ ]:
liste_hts_new = pd.ExcelWriter(f"rapportage_liste_filles_hts_{date_du_jour}.xlsx",engine="xlsxwriter")
hts_new_df.to_excel(liste_hts_new,sheet_name="Total_filles_pr_commune",index=True)
hts_new.to_excel(liste_hts_new,sheet_name="liste_des_filles_global",index=False,na_rep="NULL")
for columns,communes in zip(hts_new.actual_commune.unique(), new_communes_general):
    globals().get(f"hts_new_{name_handler(columns)}").to_excel(liste_hts_new,sheet_name=f"{name_handler(communes)}",index=False,na_rep="NULL")
liste_hts_new.save()
#liste_hts_new.close()